In [0]:
import json

with open('Lab7_summarization_dataset.txt', encoding="utf-8") as json_file:
    df = json.load(json_file)

In [2]:
df


['В Сербии арестован последний военный преступник\nЗадержан Горан Хаджич , разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах .\n\nКак сообщает « Лента.ру » , первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля , однако вскоре были опровергнуты .\nВ ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич .\nОб этом сообщает сербский телеканал B92 .\n\nХорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина — непризнанного сербского государства , существовавшего на территории Хорватии в 1992 — 1995 годах .\nВ 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности .\nВ частности , ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 — 1993 годах .\n\nЗа информацию , способную помочь в поимке Горана Хаджича , в США 

In [52]:
!pip install git+https://github.com/nlpub/pymystem3
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from itertools import combinations
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer,word_tokenize
from nltk.stem.snowball import RussianStemmer
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from pymystem3 import Mystem
from nltk.cluster.util import cosine_distance
import networkx as nx
import numpy as np
import re, string, random



def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)


lmtzr = RussianStemmer()
m = Mystem()
tokenizer = RegexpTokenizer(r'\w+')
stop_words_ru = stopwords.words('russian')
stop_words_ru.extend(['что', 'но','это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])

cleared_text=[]
cleared_sentences = []
cleared_words = []

for text in df:
  sentences_list = sent_tokenize(text)

  for sentences in sentences_list:  
    words = word_tokenize(sentences)
    for word in words:
      if (word not in stop_words_ru): #and word not in string.punctuation):
        #cleared_words.append(m.lemmatize(word))
        cleared_words.append(word)
    cleared_sentences.append(' '.join(cleared_words))
    cleared_words = []
      
  cleared_text.append(' '.join(cleared_sentences))
  cleared_sentences=[]

summarize_all_text =[]
summarize_text =[]

for text in df:
  sentences_list = sent_tokenize(text)
  sentence_similarity_martix = build_similarity_matrix(sentences_list, stop_words_ru)

  # Step 3 - Rank sentences in similarity martix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
  scores = nx.pagerank(sentence_similarity_graph)


  # Step 4 - Sort the rank and pick top sentences
  ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences_list)), reverse=True)    
  #print("Indexes of top ranked_sentence order are ", ranked_sentence)  

  top_n = round(len(sentences_list)/2)

  for i in range(top_n):
    summarize_text.append(ranked_sentence[i][1])
  
  summarize_all_text.append(summarize_text)
  summarize_text = []

with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(summarize_all_text, f, ensure_ascii=False, indent=4)



  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-65_3zp0_
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-65_3zp0_
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=67f2f66e99d8c2a2e7c21f5b4be6111a9a354ba8689e173e660904f448e6bef2
  Stored in directory: /tmp/pip-ephem-wheel-cache-kfugh7f1/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Indexes of top ranked_sentence order are  [(0.09294556955958248, 'Выдача обвиняемых в военных преступлениях является одним из условий для принятия Сербии в Европейский союз .'), (0.09282288106320352, 'Как сообщает « Лента.ру » , первые неофициальные сообщения об аресте Горана Хаджича появ